# HPXPy Multi-Locality Demo

This notebook demonstrates concepts for multi-locality (multi-process) HPX execution.

**Important:** True multi-locality execution requires running the `multi_locality_demo.py` script, which spawns multiple HPX processes. This notebook demonstrates the concepts and APIs in single-locality mode.

## What is Multi-Locality?

- **Locality**: An HPX execution context, typically one per node/process
- **Multi-Locality**: Running HPX across multiple processes that can communicate
- **Use Cases**: Distributed computing, cluster computing, HPC workloads

## Running True Multi-Locality

```bash
# Run with 2 localities (default)
python multi_locality_demo.py

# Or launch with more localities:
python -c "from hpxpy.launcher import launch_localities; launch_localities('multi_locality_demo.py', num_localities=4, verbose=True)"
```

In [ ]:
import numpy as np
import hpxpy as hpx

hpx.init(num_threads=4)

## Locality Information

In [ ]:
my_id = hpx.locality_id()
num_locs = hpx.num_localities()

print(f"Current locality ID: {my_id}")
print(f"Total localities: {num_locs}")
print(f"HPX threads: {hpx.num_threads()}")

## Local Array Operations

In [ ]:
# Each locality creates some local data
local_data = np.array([float(my_id + 1)] * 10)
local_arr = hpx.from_numpy(local_data)

print(f"[Locality {my_id}] Local data sum: {hpx.sum(local_arr)}")

## Collective Operations (Single-Locality Behavior)

These operations work in single-locality mode but become powerful in multi-locality:

In [ ]:
# Barrier synchronization
hpx.barrier("sync_point")
print(f"[Locality {my_id}] Passed barrier")

# All-reduce: sum values from all localities
global_sum = hpx.all_reduce(local_arr, op='sum')
print(f"[Locality {my_id}] Global sum: {hpx.sum(global_sum)}")
print("  (In single-locality, global_sum == local_sum)")

In [ ]:
# Broadcast from locality 0
if my_id == 0:
    broadcast_data = np.array([42.0, 43.0, 44.0])
else:
    broadcast_data = np.zeros(3)

broadcast_arr = hpx.from_numpy(broadcast_data)
received = hpx.broadcast(broadcast_arr, root=0)
print(f"[Locality {my_id}] Received broadcast: {received.to_numpy()}")

In [ ]:
# Gather to locality 0
my_contribution = np.array([float(my_id * 100 + i) for i in range(3)])
my_arr = hpx.from_numpy(my_contribution)
gathered = hpx.gather(my_arr, root=0)

if my_id == 0:
    print(f"[Locality 0] Gathered {len(gathered)} arrays:")
    for i, arr in enumerate(gathered):
        print(f"  From locality {i}: {arr}")

## Multi-Locality Programming Pattern

The typical pattern for multi-locality HPX programs:

```python
from hpxpy.launcher import spmd_main, is_multi_locality_mode

@spmd_main(num_localities=4)
def main():
    import hpxpy as hpx
    hpx.init()
    
    try:
        my_id = hpx.locality_id()
        
        # Each locality works on its portion of data
        local_data = load_local_portion(my_id)
        local_result = process(local_data)
        
        # Synchronize and combine results
        hpx.barrier("sync")
        global_result = hpx.all_reduce(local_result, op='sum')
        
        if my_id == 0:
            save_result(global_result)
    finally:
        hpx.finalize()

if __name__ == "__main__":
    if is_multi_locality_mode():
        # Spawned process - run directly
        run_computation()
    else:
        # Original process - launch multiple localities
        main()
```

## Collective Operations Summary

| Operation | Description | Communication Pattern |
|-----------|-------------|----------------------|
| `barrier(name)` | Synchronize all localities | All-to-all |
| `all_reduce(arr, op)` | Reduce and broadcast result | All-to-all |
| `broadcast(arr, root)` | Send from one to all | One-to-all |
| `gather(arr, root)` | Collect from all to one | All-to-one |
| `scatter(arr, root)` | Distribute from one to all | One-to-all |

In [ ]:
hpx.finalize()
print("\nDemo complete!")
print("For true multi-locality execution, run: python multi_locality_demo.py")